# Prediction de l'accélération et du freinage avec nuScenes

In [ ]:
%matplotlib inline

from nuscenes.nuscenes import NuScenes
from nuscenes.can_bus.can_bus_api import NuScenesCanBus

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# data/sets/nuscenes
# D:\Utilisateurs\Alexandre\Repertoire_D\nuscenes\v1 0-trainval01

#nusc = NuScenes(version='v1.0-mini', dataroot='../data/sets/nuscenes')
nusc = NuScenes(version='v1.0-trainval', dataroot='D:/Utilisateurs/Alexandre/Repertoire_D/nuscenes/v1.0-trainval01')

nusc_can = NuScenesCanBus(dataroot='../data/sets/nuscenes')

## Anticiper la vitesse avec un véhicule en face

Principe de base (idée): 
    - le système de pilotage envoie les postions gps à suivre, la vitesse et l'angle recommandés à l'algorithme de gestion
    - l'algorithme de gestion en fonction de la situation va envoyer sur un modèle pour donner une nouvelle valeur de la vitesse et de l'angle par rapport à l'environnement
    - plusieurs modèles, mais en premier un basique : s'il y a personne en face on change rien, sinon calcul de la nouvelle vitesse par rapport au véhicule devant.
Pour cela, il nous faut plusieurs données pour ce modèle:
    - position, vitesse, orientation du véhicule égo (nous) et du véhicule en face
    - à compléter?
        

### 1/ Recherche de ces données

Prenons une scene où on suit une voiture, la scène 61. Cherchons l'instance de la voiture....

In [ ]:
scene_test = nusc.scene[58] # 58 avec le dataset normal #0 avec le minidataset
scene_test

In [ ]:
sample = nusc.get('sample',scene_test['last_sample_token'])
ann = sample['anns'][1]
ann_meta = nusc.get('sample_annotation', ann)
#nusc.list_sample(sample['token'])
sample

In [ ]:
liste_vehicle = []
for at in sample['anns']:
    meta_data = nusc.get('sample_annotation',at)
    if meta_data['category_name'] == 'vehicle.car': 
        liste_vehicle += [meta_data]
nusc.render_annotation("bc3180b07f8e4a728f504ded654df56f")

Token de l'instance de la voiture que l'on suit: c1958768d48640948f6053d04cffd35b

Maintenant, comparons la position de cette voiture à la notre. `field2token` permet d'accéder à la liste de tout les enregistrements d'une scene.

In [ ]:
meta_data

In [ ]:
ann_tokens = set(nusc.field2token('sample_annotation','instance_token',"c1958768d48640948f6053d04cffd35b"))
def print_pos_rot(pos,rot):
    print("{:04.2f} {:04.2f} {:04.2f}".format(pos[0],pos[1],pos[2]))
    print("{:04.2f} {:04.2f} {:04.2f}".format(rot[0],rot[1],rot[2]))

ego_pos = []
voiture_pos = []
ego_rot = []
voiture_rot = []
for at in ann_tokens:
    meta_data = nusc.get('sample_annotation',at)
    sample = nusc.get('sample',meta_data['sample_token'])
    timestamp = sample['timestamp']
    lidar = nusc.get('sample_data',sample['data']['LIDAR_TOP'])
    ego_token = lidar['ego_pose_token']
    ego = nusc.get('ego_pose',ego_token)
    #print_pos_rot(ego['translation'],ego['rotation'])
    #nt_pos_rot(meta_data['translation'],meta_data['rotation'])
    #print(meta_data['rotation'])
    ego_pos += [ego['translation']]
    ego_rot += [ego['rotation']]
    voiture_pos += [meta_data['translation']]
    voiture_rot += [meta_data['rotation']]    

#print(nusc.ego_pose[:300])
print(nusc.ego_pose[0])
len(nusc.ego_pose)

Maintenant qu'on a les données de position de notre véhicule et celui d'en face, on peut essayer de visualiser sur la map ce qu'il en est pour pouvoir déterminer une formule/ fonction pour savoir si une voiture est en face de nous (sur la même voie serait l'idéal.
Problème, la rotation est en écriture quaternion (je ne sais pas comment manipuler), donc en attendant je vais essayer de détecter le véhicule en face par rapport à s'il est détecté par la caméra frontale puis en calculant la distance.


In [ ]:
df_ego_pos = pd.DataFrame(ego_pos,columns=["x","y","z"])
#print(ego_rot)
#df_ego_rot = pd.DataFrame(ego_rot,columns=["x","y","z"])
df_voiture_pos = pd.DataFrame(voiture_pos,columns=["x","y","z"])
#df_voiture_rot = pd.DataFrame(voiture_rot,columns=["x","y","z"])
plt.plot(df_ego_pos["x"],df_ego_pos["y"],'bo')
plt.plot(df_voiture_pos["x"],df_voiture_pos["y"],'ro')


### 2/ Pretraitement sur les données
En regardant le code du sdk j'ai trouvé un bout de code utile qui me permet de savoir quel caméra a détecté l'annotation. Pour faire simple, je vais considérer (actuellement, cela va peut-être changer), qu'une voiture est en face de la notre si elle est détectée seulement par la caméra frontale et qu'elle respecte une certaine distance. Je vais maintenant regrouper toute ces données où on a une voiture en face nous.

`get_sample_data` renvoie plusieurs informations dont une liste de boxe de la caméra, si on lui passe un token en paramètre, il renvoie une seul boxe si l'instance est capturé par la caméra, rien sinon.

On peut maintenant créer le dataframe contenant toutes les informations: vitesse,distance,timestamp,accélération, token....
Voir `vehicle_data.csv`

In [ ]:
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility

# renvoie vrai et un un tableau rempli si l'instance est en face d'ego et ego ne tourne pas trop
def find_vehicle_in_front(instance_token):
    instance = nusc.get('instance',instance_token)
    last_token = instance["last_annotation_token"]
    curr_token = instance["first_annotation_token"]
    rows_list = []
    i = 0
    
    ann = nusc.get('sample_annotation',instance["first_annotation_token"])
    sample = nusc.get('sample',ann['sample_token'])
    scene = nusc.get('scene',sample['scene_token'])
    dict_scene = nusc_can.get_messages(scene['name'],'vehicle_monitor') 
    taille = len(dict_scene)
    # Traitement sur tout les vehicules ou non (a faire)
    all_camera = False
    if all_camera:
        list_cams = [key for key in sample_record['data'].keys() if 'CAM' in key]
    else:
        list_cams = ['CAM_FRONT']
        
    # Pour chaque enregistrement de l'annoation on ajoute une ligne avec les elements
    while curr_token != last_token:
        curr_ann = nusc.get('sample_annotation',curr_token)
        curr_sample = nusc.get('sample',curr_ann['sample_token'])        
        cams_check = []
        # (abs(dict_scene[i]['utime'] - curr_sample['timestamp']) > 250000 ) and
        while i < taille - 2 and   (dict_scene[i]['utime'] + 250000 < curr_sample['timestamp']):
            #print("avance: ",i,curr_sample['timestamp']-1532402900000000," ",dict_scene[i]['utime']-1532402900000000)
            i += 1
        #print(curr_sample['timestamp'] - 1532402900000000,dict_scene[i]['utime'] - 1532402900000000  )  
        # récupérer les caméras qui ont vu l'annotation

        _, boxes, _ = nusc.get_sample_data(curr_sample['data']['CAM_FRONT'], box_vis_level=BoxVisibility.ANY,
                                                selected_anntokens=[curr_token])
        #and abs(dict_scene[i]['steering']) < 100
        if len(boxes) > 0:
            #calcul distance entre ego et le vehicule
            lidar = nusc.get('sample_data',curr_sample['data']['LIDAR_TOP'])
            ego = nusc.get('ego_pose',lidar['ego_pose_token'])
            dist = np.linalg.norm(np.array(ego['translation']) - np.array(curr_ann['translation']))
            ego_pos = [round(e,3) for e in ego['translation']]
            object_pos = [round(e,3) for e in curr_ann['translation']]

            dic = {'scene':scene['name'],'timestamp':curr_sample['timestamp'],'utime':dict_scene[i]['utime'],'inst_token':instance_token,
                   'ann_token':curr_token,'ego_pos':ego_pos,'object_pos':object_pos,
                   'distance':round(dist,3),'steering':round(dict_scene[i]['steering'],3),'ego_speed':round(dict_scene[i]['vehicle_speed'],3),'throttle':dict_scene[i]['throttle'],
                   'brake':dict_scene[i]['brake'],'future_throttle':dict_scene[i+1]['throttle'],'future_brake':dict_scene[i+1]['brake']}
            rows_list += [dic]
                
        curr_token = curr_ann['next']   
        if i < taille - 2:
            i += 1
    #print(len(rows_list),len(dict_scene))
    return len(rows_list)!=0,rows_list

# renvoie un tableau rempli pour toute la scene avec les données du véhicule
def vehicle_info(scene):
    #sample = nusc.get('sample',scene['first_sample_token'])
    curr_token = scene['first_sample_token']
    rows_list = []
    last_token = scene['last_sample_token']
    i = 0

    dict_scene = nusc_can.get_messages(scene['name'],'vehicle_monitor') 
    taille = len(dict_scene)
    
    # Pour chaque enregistrement de la scene on ajoute une ligne avec les elements
    while curr_token != last_token:
        curr_sample = nusc.get('sample',curr_token)        
        while i < taille -2 and  (dict_scene[i]['utime'] + 250000 < curr_sample['timestamp']):
            i += 1
        #print(curr_sample['timestamp'] - 1532402900000000,dict_scene[i]['utime'] - 1532402900000000  )  
        lidar = nusc.get('sample_data',curr_sample['data']['LIDAR_TOP'])
        ego = nusc.get('ego_pose',lidar['ego_pose_token'])
        ego_pos = [round(e,3) for e in ego['translation']]
        dic = {'scene':scene['name'],'timestamp':curr_sample['timestamp'],'utime':dict_scene[i]['utime'],'inst_token':"vehicle_info",
               'ann_token':curr_token,'ego_pos':ego_pos,'object_pos':ego_pos,
               'distance':99,'steering':round(dict_scene[i]['steering'],3),'ego_speed':round(dict_scene[i]['vehicle_speed'],3),'throttle':dict_scene[i]['throttle'],
              'brake':dict_scene[i]['brake'],'future_throttle':dict_scene[i+1]['throttle'],'future_brake':dict_scene[i+1]['brake']}
        rows_list += [dic]
        curr_token = curr_sample['next']   
        if i < taille - 2:
            i += 1
    #print(len(rows_list),len(dict_scene))
    return rows_list
_ ,dic = find_vehicle_in_front("c1958768d48640948f6053d04cffd35b")
#print(dic)
df = pd.DataFrame.from_dict(dic).sort_values(by='timestamp').reset_index(drop=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)

In [ ]:
#nusc_can.list_misaligned_routes()
blackint = nusc_can.can_blacklist
blacklist = [ "scene-0"+ str(i) for i in blackint]
blacklist

In [ ]:
blackint = nusc_can.can_blacklist
blacklist = [ "scene-0"+ str(i) for i in blackint]
# 1532402936198962 1532402936699359 1532402937198682

# Liste toutes les instances d'une scene
def get_instances_scene(scene):
    sample = nusc.get('sample',scene['first_sample_token'])
    list_instances = []
    while sample['token'] != scene['last_sample_token']:
        anns = sample['anns']
        for ann_token in anns:
            ann = nusc.get('sample_annotation',ann_token)
            instance = nusc.get('instance',ann['instance_token'])
            category = nusc.get('category',instance['category_token'])
            if not instance in list_instances and "vehicle" in category['name']:
                list_instances += [instance]
        sample = nusc.get('sample',sample['next'])
    return list_instances

# Renvoie un dataframe contenant les données de toutes les instances de la scene
def build_dataframe_for_one_scene(s,affichage):
    list_rows = vehicle_info(s)
    list_instances = get_instances_scene(s)
    for inst in list_instances:
        ok, res = find_vehicle_in_front(inst['token'])
        if affichage:
            print(len(res)," echantillons")   
        if ok:
            list_rows += res
    return pd.DataFrame.from_dict(list_rows).sort_values(by='timestamp').reset_index(drop=True)

# Explore chaque scene, puis chaque instance de cette scene qui est un vehicle en mouvement (devant)
# Cree un dataframe avec pour entree distance au vehicle, ego_vitesse, ego_accel, ego_brake 
# et vehicle_vitesse (pas mtn)
def build_dataframe_for_vehicle_in_front():
    scenes = nusc.scene
    list_rows = []
    first = True
    i = 0
    for s in scenes[:20]:
        if s['name'] not in blacklist and s['name']  not in ["scene-0003","scene-0419"]:
            #print(s['name'])
            if i % 100 == 0:
                print("#",end='')
            if first:
                dataframe = build_dataframe_for_one_scene(s,False)
                first = False
            else:
                datatemp = build_dataframe_for_one_scene(s,False)
                dataframe = dataframe.append(datatemp,ignore_index=True)
            i += 1

    print(dataframe)
    print(dataframe.describe())
    return dataframe
    
#find_vehicle_in_front("c1958768d48640948f6053d04cffd35b")
# 15k ligne sans contrainte sur steering (100 scenes)
df_vehicle = build_dataframe_for_vehicle_in_front()
df_vehicle = df_vehicle.sort_values(by=['timestamp'])
#df_vehicle.to_csv(path_or_buf="./data/vehicle_data_30juin_full.csv",index=False)
df_brake = df_vehicle[['scene','brake']]
#df_brake.to_csv('./data/vehicle_brake_25j.csv')
scene_name = 'scene-0061'
my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
scene = nusc.get('scene',my_scene_token)
#nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')
df = build_dataframe_for_one_scene(scene,False)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
display(df)

On faisant une matrice de corrélation, on aperçoit que la distance entre les deux véhicules et un influencer par la vitesse principalement, le frein agit un peu dessus. De plus, la vitesse est trés corrélée avec le frein. L'accélération a un peu d'effet sur la vitesse, voir aucun sur la distance, cela est un peu étonnant.

In [ ]:
print(df_vehicle.shape)
df_vehicle.corr()

Scène intéressante pour prédire le frein: 14 
Scène avec piéton: 25,29,43

Il pourrait être intéressant de pouvoir afficher la scène avec les résultats de la prédiction

In [ ]:
scene_name = 'scene-0067'
my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
#nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')

In [ ]:
def sort_distance(dataframe):
    pass

In [ ]:
import cv2
from typing import Tuple, List
import os.path as osp
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix
import operator

# parametres pour cv2
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (0,500)
fontScale              = 1
fontColor              = (255,255,255)
color              = (255,0,0)
lineType               = 2
pas = (0,50)

def get_color(category_name: str) -> Tuple[int, int, int]:
    """
    Provides the default colors based on the category names.
    This method works for the general nuScenes categories, as well as the nuScenes detection categories.
    """
    if 'bicycle' in category_name or 'motorcycle' in category_name:
        return 255, 61, 99  # Red
    elif 'vehicle' in category_name or category_name in ['bus', 'car', 'construction_vehicle', 'trailer', 'truck']:
        return 255, 158, 0  # Orange
    elif 'pedestrian' in category_name:
        return 0, 0, 230  # Blue
    elif 'cone' in category_name or 'barrier' in category_name:
        return 0, 0, 0  # Black
    else:
        return 255, 0, 255  # Magenta

def affichage(im,df_curr):
           
    cv2.putText(im, 'Vitesse:'+ str(df_curr.iat[0,9]), 
        bottomLeftCornerOfText, 
        font, 
        fontScale, 
        fontColor,
        lineType)
    cv2.putText(im, 'Angle volant:'+ str(df_curr.iat[0,8]), 
        tuple(map(operator.add, bottomLeftCornerOfText,(0,50))), 
        font, 
        fontScale, 
        fontColor,
        lineType)
    cv2.putText(im, 'Acceleration:'+ str(df_curr.iat[0,10]), 
        tuple(map(operator.add, bottomLeftCornerOfText,(0,100))), 
        font, 
        fontScale, 
        fontColor,
        lineType)

    cv2.putText(im, 'Frein:'+ str(df_curr.iat[0,11]), 
        tuple(map(operator.add, bottomLeftCornerOfText,(0,150))), 
        font, 
        fontScale, 
        fontColor,
        lineType)
    cv2.putText(im, 'Acceleration (Pred):'+ str(df_curr.iat[0,12]), 
        tuple(map(operator.add, bottomLeftCornerOfText,(0,200))), 
        font, 
        fontScale, 
        fontColor,
        lineType)

    cv2.putText(im, 'Frein (Pred):'+ str(df_curr.iat[0,11]), 
        tuple(map(operator.add, bottomLeftCornerOfText,(0,250))), 
        font, 
        fontScale, 
        fontColor,
        lineType)

    if df_curr.shape[0] > 1:
        cv2.putText(im, 'Distance:'+ str(df_curr.iloc[1]['distance']), 
            tuple(map(operator.add, bottomLeftCornerOfText,(0,300))), 
            font, 
            fontScale, 
            color,
            lineType)    
        
def draw_rect(im,selected_corners, color):
    prev = selected_corners[-1]
    for corner in selected_corners:
        cv2.line(im,
                 (int(prev[0]), int(prev[1])),
                 (int(corner[0]), int(corner[1])),
                 color, 2)
        prev = corner
        
def render_scene_channel_with_predict(nusc,
                        scene_token: str, dataframe,
                        channel: str = 'CAM_FRONT',
                        freq: float = 10,
                        imsize: Tuple[float, float] = (960, 540),
                        out_path: str = None) -> None:
    """
    Renders a full scene for a particular camera channel.
    :param scene_token: Unique identifier of scene to render.
    :param channel: Channel to render.
    :param freq: Display frequency (Hz).
    :param imsize: Size of image to render. The larger the slower this will run.
    :param out_path: Optional path to write a video file of the rendered frames.
    Reprise de la fonction render_scene_channel du sdk nuscenes mais avec beaucoup de changements
    Renvoie un dataframe avec un objet par timestamp 
    """
    valid_channels = ['CAM_FRONT_LEFT', 'CAM_FRONT', 'CAM_FRONT_RIGHT',
                        'CAM_BACK_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT']

    assert imsize[0] / imsize[1] == 16 / 9, "Aspect ratio should be 16/9."
    assert channel in valid_channels, 'Input channel {} not valid.'.format(channel)

    if out_path is not None:
        assert osp.splitext(out_path)[-1] == '.avi'

    # Get records from DB
    scene_rec = nusc.get('scene', scene_token)
    sample_rec = nusc.get('sample', scene_rec['first_sample_token'])
    sd_rec = nusc.get('sample_data', sample_rec['data'][channel])

    # Open CV init
    name = '{}: {} (Space to pause, ESC to exit)'.format(scene_rec['name'], channel)
    cv2.namedWindow(name)
    cv2.moveWindow(name, 0, 0)

    if out_path is not None:
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        out = cv2.VideoWriter(out_path, fourcc, freq, imsize)
    else:
        out = None

    # parametres pour cv2
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,500)
    fontScale              = 1
    fontColor              = (255,255,255)
    color              = (255,0,0)
    lineType               = 2
    pas = (0,50)
    # parametres pour afficher infos
    i = 0
    taille = dataframe.shape[0]
    scene_token = nusc.field2token('scene', 'name', dataframe.at[0,'scene'])[0]
    scene = nusc.get('scene',scene_token)
    sample = nusc.get('sample',scene['first_sample_token'])
    df_curr = dataframe[dataframe['timestamp'] == sample['timestamp']]
    df_curr = df_curr.sort_values(by='distance').reset_index(drop=True)
    #print(df_curr)
    has_more_frames = True
    angle = df_curr.iat[0,8]
    xmin = 10
    xmax = - 10
    colors: Tuple = ((0, 0, 255), (255, 0, 0), (155, 155, 155))
    borne_a = 600
    borne_b = 1000
    new_df = pd.DataFrame(columns=df_curr.columns)
    while has_more_frames:
        ann = df_curr[df_curr["inst_token"]=="98300b9c4acb4da9a7aecd0084650265"]
        ann_tok = ann['ann_token']
        #print(df_curr['inst_token'])
        # selected_anntokens=[ann_tok.iat[0]]
        # Get data from DB
        impath, boxes, camera_intrinsic = nusc.get_sample_data(sd_rec['token'],
                                                                    box_vis_level=BoxVisibility.ANY)
        # Load and render
        if not osp.exists(impath):
            raise Exception('Error: Missing image %s' % impath)
        im = cv2.imread(impath)
        dmin = 999
        minbox = None
        df_wv = df_curr[df_curr["inst_token"]!="vehicle_info"]
        liste = []
        annmin = "none"
        for j in range(df_wv.shape[0]):
            ann = df_wv.iloc[j]['ann_token']
            impath, boxes, camera_intrinsic = nusc.get_sample_data(sd_rec['token'],
                                                            box_vis_level=BoxVisibility.ANY,
                                                            selected_anntokens=[ann])
            if len(boxes) != 0:
                liste += [(ann,boxes[0])]

        # Pour chaque couple annotation/box on cherche le plus prés d'ego en face
        for (a,box) in liste:
            corners = view_points(box.corners(), camera_intrinsic, normalize=True)[:2, :]
            if (box.center[2] < dmin and corners.T[4][0] < borne_b-angle and corners.T[6][0] > borne_a-angle 
                and "vehicle" in box.name):
                dmin = box.center[2]
                annmin = a
                minbox = box
                
        if dmin != 999:
            c = get_color(minbox.name)
            corners = view_points(minbox.corners(), camera_intrinsic, normalize=True)[:2, :]
            draw_rect(im,corners.T[4:], colors[1][::-1])
            
        if i%6 == 0 and i != 0:
            if dmin != 999:
                new_df = new_df.append(df_curr[df_curr['ann_token']==annmin])
            else:
                new_df = new_df.append(df_curr[df_curr['inst_token']=="vehicle_info"])


        # Affichage informations
        if sample['token'] != scene['last_sample_token']:
            if not df_curr.empty:
                if dmin != 999:
                    cv2.line(im, (int((corners.T[4][0]+corners.T[6][0])/2), 400), (int((corners.T[4][0]+corners.T[6][0])/2), 600), (255, 255, 0), thickness=2)
                    cv2.putText(im, 'Center:'+ str(round(minbox.center[0],3))+"\n      "+str(round(minbox.center[2],2)), 
                                (int(800+minbox.center[0]*10),250), 
                                font, 
                                fontScale, 
                                (255, 0, 255),
                                lineType)
                cv2.line(im, (int(borne_b-angle), 400), (int(borne_b-angle), 600), (255, 0, 0), thickness=2)
                cv2.line(im, (int(borne_a-angle), 400), (int(borne_a-angle), 600), (255, 0, 0), thickness=2)
                affichage(im,df_curr)
            else:
                print(sample['timestamp'])
            if i%6 == 0 and i != 0:
                sample = nusc.get('sample',sample['next'])
                df_curr = dataframe[dataframe['timestamp'] == sample['timestamp']]
                df_curr = df_curr.sort_values(by='distance').reset_index(drop=True)
                if not df_curr.empty:
                    angle = df_curr.iat[0,8]
                #angle = 0
        else:
            print("fin des données ",i)

            
        # Render
        im = cv2.resize(im, imsize)
        cv2.imshow(name, im)
        if out_path is not None:
            out.write(im)

        key = cv2.waitKey(1)  # Images stored at approx 10 Hz, so wait 10 ms.
        if key == 32:  # If space is pressed, pause.
            key = cv2.waitKey()

        if key == 27:  # if ESC is pressed, exit
            cv2.destroyAllWindows()
            break

        if not sd_rec['next'] == "":
            sd_rec = nusc.get('sample_data', sd_rec['next'])
        else:
            has_more_frames = False
        i += 1
    print("nombre de frame: ",i)
    new_df = new_df.reset_index(drop=True)

    #print(new_df)
    cv2.destroyAllWindows()
    if out_path is not None:
        out.release()
    return new_df


In [ ]:
scene_name = 'scene-0120'
my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
scene = nusc.get('scene',my_scene_token)
df = build_dataframe_for_one_scene(scene,False)
render_scene_channel_with_predict(nusc,my_scene_token,df,'CAM_FRONT')
print()

In [ ]:
scene_name = 'scene-0160'
my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
scene = nusc.get('scene',my_scene_token)
#nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')
#df = build_dataframe_for_one_scene(scene,False)
#print(df)
#df.to_csv(scene_name+".csv")
liste_scene = ['scene-0061','scene-0010','scene-0041','scene-0009','scene-0100',
'scene-0101','scene-0190','scene-0194']
#liste_scene = ['scene-0108']
# render_scene_channel_with_predict(nusc,my_scene_token,df, 'CAM_FRONT')
df_for_predict = pd.DataFrame(columns=df.columns)
for s in liste_scene:
    s_token = nusc.field2token('scene', 'name', s)[0]
    scene = nusc.get('scene',s_token)
    df = build_dataframe_for_one_scene(scene,False)
    temp = render_scene_channel_with_predict(nusc,s_token,df, 'CAM_FRONT',imsize=(256,144))
    df_for_predict = df_for_predict.append(temp)
    
df_for_predict.to_csv("./data/df_predict_throttle_brake.csv")
    



### 3/ Apprentissage


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm, neighbors
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import linear_model
import random

In [ ]:
df_for_predict = df_for_predict.reset_index(drop=True)
# ["distance","ego_speed","throttle","brake"]
features = ["distance","ego_speed","throttle","brake"]
features = ["distance","ego_speed"]
X = df_vehicle[features]
y_throttle = df_vehicle["future_throttle"]
y_brake = df_vehicle["future_brake"]
Xt_train, Xt_test, yt_train, yt_test = train_test_split(X,y_throttle, test_size = 0.2, random_state = 1)
Xb_train, Xb_test, yb_train, yb_test = train_test_split(X,y_brake, test_size = 0.2, random_state = 1)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    display(Xt_train)

In [ ]:
# temp
features = ["distance","ego_speed"]
#features = [col for col in train_data.columns.to_list() if ('distance' in col or'ego_speed' in col)]
print(features)
y_throttle = df_for_predict["throttle"]
y_brake = df_for_predict["brake"]
X = df_for_predict[features]
Xt_train, Xt_test, yt_train, yt_test = train_test_split(X,y_throttle, test_size = 0.2, random_state = 1)
Xb_train, Xb_test, yb_train, yb_test = train_test_split(X,y_brake, test_size = 0.2, random_state = 1)

In [ ]:
model_t = svm.SVR()
model_t.fit(Xt_train,yt_train)
print(model_t.score(Xt_test,yt_test))
model_b = svm.SVR()
model_b.fit(Xb_train,yb_train)
print(model_b.score(Xb_test,yb_test))
model_b = RandomForestRegressor(n_estimators=100,random_state=0)
model_b.fit(Xb_train,yb_train)
print(model_b.score(Xb_test,yb_test))

In [ ]:
df_full = pd.read_csv("./data/vehicle_data_30juin_full.csv")

In [ ]:
%matplotlib inline

plt.plot(df_for_predict['distance'],df_for_predict['brake'],'bo')

#plt.plot(df_full['distance'],df_full['brake'],'bo')
plt.xlim(0, 50)

plt.show()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D  
%matplotlib qt

def randrange(n, vmin, vmax):
    '''
    Helper function to make an array of random numbers having shape (n, )
    with each number distributed Uniform(vmin, vmax).
    '''
    return (vmax - vmin)*np.random.rand(n) + vmin
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
n = 100

ax.scatter(df_vehicle['ego_speed'],df_vehicle['brake'],df_vehicle['distance'],marker='o')
ax.set_xlabel('V')
ax.set_ylabel('Brake')
ax.set_zlabel('Distance')

plt.show()

67,185,130,155 piéton traverse,
6 camion au milieu (pas mal), 41 on s'arrete devant une voiture a un feu (genial), 190
9,10 il se passe rien
14 arret puis démarrage car bus avance
45 à exclure 

Prédire trajectoire en entier...

In [ ]:
df_ego = df[df['inst_token'] == "vehicle_info"]
#df_ego

list_vec = [(df_ego.iloc[i+1]['ego_pos'][0] - df_ego.iloc[i]['ego_pos'][0],
            df_ego.iloc[i+1]['ego_pos'][1] - df_ego.iloc[i]['ego_pos'][1]) 
            for i in range(df_ego.shape[0]-1) ]
list_vitesse = [df_ego.iloc[i]['ego_speed'] for i in range(df_ego.shape[0]-1)]
list_vec_norm = [ (v[0]/np.sqrt((v[0]*v[0] + v[1]*v[1])),v[1]/np.sqrt((v[0]*v[0] + v[1]*v[1])))
            for v in list_vec ]
#print(list_vec)
list_vec_norm

for i in range(df_ego.shape[0]-1):
    # tuple(map(operator.add, df_ego.iloc[i]['ego_pos'],
    r = [e * list_vitesse[i]/3.6*0.5 for e in list_vec_norm[i]]
    #print(list_vec_norm[i])
    new_pos =   list(map(operator.add, df_ego.iloc[i]['ego_pos'],r))
    new_pos = [round(e,3) for e in new_pos]    
    #print(new_pos,df_ego.iloc[i+1]['ego_pos'])


In [ ]:
# Premiere version , ne marche pas
def compute_distance(pos,ABn,dataframe):
    #dist = np.linalg.norm(np.array(ego['translation']) - np.array(curr_ann['translation']))
    dataframe = dataframe.drop(columns=['distance'])
    taille = dataframe.shape[0]
    dmin = 99
    mini = 0
    if ABn[0] == 0:
        a = 0
    else:
        a = ABn[1]/ABn[0]
    c = - pos[0] * a + pos[1]    
    for i in range(taille):
        row = dataframe.iloc[i]
        if row["inst_token"] != "vehicle_info":
            distance_ego = round(np.linalg.norm(np.array(pos) - np.array(row['object_pos'][:2])),3)
            distance_vecteur_vitesse = np.absolute(row['object_pos'][1] - a * row['object_pos'][0] - c)/ np.sqrt(a*a + 1)
            if distance_ego < dmin and distance_vecteur_vitesse < 5:
                dmin = distance_ego
                mini = i
    #print("Distance:",dmin," ",dataframe.iloc[mini]['inst_token']," ",dataframe.iloc[mini]['object_pos'])
    return dmin
    
    #ego_pos = [round(e,3) for e in ego['translation']]
    #object_pos = [round(e,3) for e in curr_ann['translation']]
# version simplifie pour pouvoir tester 
def compute_distance_cheat(pos,ABn,dataframe):
    df = dataframe[dataframe['inst_token']=="98300b9c4acb4da9a7aecd0084650265"]
    if df.shape[0] == 0:
        return 99
    
    return round(np.linalg.norm(np.array(pos) - np.array(df.iloc[0]['object_pos'][:2])),3)

# Fonction qui déroule une scene en se basant sur les predictions faites, 
# Point de départ = pos initial puis après calcul à chaque tour de boucle par rapport aux retours des modèles
def predict_scene_v2(scene_name,k_past_data):
    my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
    scene = nusc.get('scene',my_scene_token)
    #nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')
    df = build_dataframe_for_one_scene(scene,False)
    df_ego = df[df['inst_token'] == "vehicle_info"]
    
    # Initialisation des paramètres
    speed = df_ego.iloc[0]['ego_speed']
    A = df_ego.iloc[0]['ego_pos'][:2]
    B = df_ego.iloc[1]['ego_pos'][:2]
    AB =   [round(B[0] - A[0],3),round(B[1] - A[1],3)]
    ABn = round(AB[0]/np.sqrt((AB[0]*AB[0] + AB[1]*AB[1])),3),round(AB[1]/np.sqrt((AB[0]*AB[0] + AB[1]*AB[1])),3)
    #print(A,B,AB,ABn)
    log = []
    features = ["distance","ego_speed","throttle","brake"]
    sample = nusc.get('sample',scene['first_sample_token'])
    last = scene['last_sample_token']
    i = 0
    throttle = [0]
    brake = [0]
    
    past_data = [0]*k_past_data*2
    print("Position Predite,    Position Reel,    Distance, vitesse,  accélération,  freinage")
    # Boucle
    while i != 30 and sample['token'] != last:
        speed = round(speed,3)
        distance = compute_distance(A,ABn,df[df['timestamp']==sample['timestamp']])
        data = [[distance,speed]]
        #data = [[distance,speed]+past_data]
        #print(data)
        throttle = model_t.predict(data)
        brake = model_b.predict(data)
        #throttle = [0]
        #brake = [0]
        
        if throttle[0] < 0:
            throttle[0] = 0.0
        if brake[0] < 0:
            brake[0] = 0.0


        #throttle = 0
        #brake = 0
        speed = speed  - 0.5
        if speed < 0:
            speed = 0 
        # Calcul nouveau point
        A = B
        deplacement = [e * speed/3.6*0.5 for e in ABn]
        #B = list(map(operator.add, B,deplacement))
        i += 1
        B = df_ego.iloc[i]['ego_pos'][:2]
        B = [round(b,3) for b in B]
        sample = nusc.get('sample',sample['next'])
        AB =   [round(B[0] - A[0],3),round(B[1] - A[1],3)]
        if AB[0]*AB[0] + AB[1]*AB[1] != 0:
            ABn = round(AB[0]/np.sqrt((AB[0]*AB[0] + AB[1]*AB[1])),3),round(AB[1]/np.sqrt((AB[0]*AB[0] + AB[1]*AB[1])),3)
        else:
            ABn = (0,0)
        log += [ABn]
        past_data.append(distance)
        past_data.append(speed)
        past_data.pop(0)
        past_data.pop(0)
        print(B,df_ego.iloc[i]['ego_pos'][:2],distance,speed,throttle,brake)
        #print(past_data)

    return log
scene_name = 'scene-0006'    
log = predict_scene_v2(scene_name,0)
#37efe5932d7f4084ac8ed3bbb5ed6220  56a9802dbe824219992d60debdea6646   23e14d768eab400099c816a3ac0ff041 98300b9c4acb4da9a7aecd0084650265       
#nusc.render_instance("98300b9c4acb4da9a7aecd0084650265")
print(len(log))
print(log)

# Brouillon / animation / recherche de données

In [ ]:
#test
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib qt
TWOPI = 2*np.pi

#init
scene_name = 'scene-0006'    
my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
scene = nusc.get('scene',my_scene_token)
#nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')
df = build_dataframe_for_one_scene(scene,False)
df_ego = df[df['inst_token'] == "vehicle_info"]



fig, ax = plt.subplots()
plt.xlim(0, 2000)
plt.ylim(0,2000)
t = np.arange(0.0, TWOPI, 0.001)
s = np.sin(t)
#l = plt.plot(t, s)

pos = df_ego[['ego_pos','ego_speed']]
#ax = plt.axis([0,TWOPI,-1,1])
v = df[df['inst_token']=="98300b9c4acb4da9a7aecd0084650265"]['object_pos']
print(pos.shape,v.shape)
print(pos)
#print(v)
ax.set_xlim([600,800])
ax.set_ylim([1400,1600])
ego, = plt.plot(0, 0, 'bo')
near_vehicle, = plt.plot(0,0,'ro')
print(ego)
text = plt.text(100,100,"oo")
vec_vitesse, = plt.plot(0,0,'k-')
def animate(i):
    ego.set_data(pos.iloc[i,0][0], pos.iloc[i,0][1])
    if i < 22:
        near_vehicle.set_data(v.iloc[i][0],v.iloc[i][1])
    text.set_text(pos.iloc[i,1])
    text.set_x(pos.iloc[i,0][0])
    text.set_y(pos.iloc[i,0][1])
    vec_vitesse.set_data([pos.iloc[i,0][0], pos.iloc[i,0][0]+log[i][0]*10], [pos.iloc[i,0][1],pos.iloc[i,0][1]+log[i][1]*10])
    #vec_vitesse.set_data([100, 100+log[i][0]*100], [100,100+log[i][1]*100])

    #print(i)

    return ego,text,near_vehicle,vec_vitesse,

# create animation using the animate() function
myAnimation = animation.FuncAnimation(fig, animate, frames=39, 
                                      interval=100, blit=True, repeat=True)

plt.show()

In [ ]:
#
scene_name = 'scene-0006'
my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
scene = nusc.get('scene',my_scene_token)
df_scene = build_dataframe_for_one_scene(scene,False)
df = df_scene
#display(df_scene)
liste_temps = sorted(set(df_scene['timestamp'].to_list()))
#liste_temps = np.sort(np.unique(df_scene['timestamp'].to_numpy()))
print(liste_temps)
list_pos = df[df['timestamp']==1531884156948944]['object_pos'].to_list()
print(list_pos)
a = np.transpose(np.asarray(list_pos))
df[(df['timestamp']==1531884156948944) & (df['inst_token']!='vehicle_info')]


In [ ]:
%matplotlib qt
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
#init
scene_name = 'scene-0006'    
my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
scene = nusc.get('scene',my_scene_token)
#nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')
df = build_dataframe_for_one_scene(scene,False)
df_ego = df[df['inst_token'] == "vehicle_info"]
liste_temps = sorted(set(df['timestamp'].to_list()))

#plt.style.use(['dark_background'])
fig, ax = plt.subplots()
mini = df['ego_pos'].min()
maxi = df['ego_pos'].max()
plt.xlim(mini[0]-50, maxi[0]+50)
plt.ylim(mini[1]-50, maxi[1]+50)
t = np.arange(0.0, TWOPI, 0.001)

pos = df_ego[['timestamp','ego_pos','ego_speed']]

#Param
ego, = plt.plot(0, 0, 'o')
near_vehicle, = plt.plot(0,0,'o')
text_ego = plt.text(100,100,"")
text_dist = plt.text(100,100,"_ego")
vec_vitesse, = plt.plot(0,0,'-')
pmin, = plt.plot(440,1100,'o')
fct, = plt.plot(0,0,'-')

def init():
    return []


def animate(frame,arg):
    i = arg[frame]
    ego_pos = pos[pos['timestamp']==i]['ego_pos'].iloc[0]
    ego.set_data(ego_pos[0],ego_pos[1])
       
    list_pos = df[(df['timestamp']==i)&(df['inst_token']!='vehicle_info')]['object_pos'].to_list()
    if len(list_pos)!=0 :
        #print(list_pos)
        a = np.transpose(np.asarray(list_pos))
        near_vehicle.set_data(a[0],a[1])
    
    text_ego.set_text(pos[pos['timestamp']==i].iloc[0,2])
    text_ego.set_x(pos[pos['timestamp']==i].iloc[0,1][0])
    text_ego.set_y(pos[pos['timestamp']==i].iloc[0,1][1])
    
    if frame < len(arg) - 2:
        j = arg[frame+1]
        A = pos[pos['timestamp']==i].iloc[0,1]
        B = pos[pos['timestamp']==j].iloc[0,1]
        AB =   [round(B[0] - A[0],3),round(B[1] - A[1],3)]
        ABn = round(AB[0]/np.sqrt((AB[0]*AB[0] + AB[1]*AB[1])),3),round(AB[1]/np.sqrt((AB[0]*AB[0] + AB[1]*AB[1])),3)
        vec_vitesse.set_data([A[0], A[0]+ABn[0]*10], [A[1],A[1]+ABn[1]*10])
        #vec_vitesse.set_data([100, 100+log[i][0]*100], [100,100+log[i][1]*100])
        if len(list_pos)!=0 :
            a = ABn[1]/ABn[0]
            c = - A[0] * a + A[1]
            dmin = 50
            posmin = []
            for p in list_pos:
                d = np.absolute(p[1] - a * p[0] - c)/ np.sqrt(a*a + 1)
                distance_ego = round(np.linalg.norm(np.array(A) - np.array(p)),3)
                if distance_ego < dmin and d < 5:
                    dmin = distance_ego
                    posmin = p
            if posmin != []:
                pmin.set_data(posmin[0],posmin[1])
                text_dist.set_text(dmin)
                text_dist.set_x(posmin[0])
                text_dist.set_y(posmin[1])        
            #print(dmin,posmin)
            x = np.linspace(mini[0]-50, maxi[0]+50)
            y = a*x + c
            fct.set_data(x,y)
            
    #print(i)

    return ego,text_ego,text_dist,near_vehicle,vec_vitesse,pmin,fct,

# create animation using the animate() function
myAnimation = animation.FuncAnimation(fig, animate, frames=len(liste_temps),fargs=([liste_temps]),
                                      interval=100, blit=True, repeat=True)

plt.show()

In [ ]:
blackint = nusc_can.can_blacklist
blacklist = [ "scene-0"+ str(i) for i in blackint]

def compute_near_vehicle_dataframe_one_scene(df):
    liste_temps = sorted(set(df['timestamp'].to_list()))
    new_df = pd.DataFrame(columns=df.columns)

    def compute_one_sample(frame):
        tstp = liste_temps[frame]
        #ego_pos = df[(df['timestamp']==tstp)&(df['inst_token']=='vehicle_info')]['ego_pos'].iloc[0]
        df_curr = df[(df['timestamp']==tstp)&(df['inst_token']!='vehicle_info')]
        if len(list_pos)!=0 :
            a = np.transpose(np.asarray(list_pos))
        if frame < len(liste_temps) - 2:
            tstp2 = liste_temps[frame+1]
            A = df[(df['timestamp']==tstp)&(df['inst_token']=='vehicle_info')]['ego_pos'].iloc[0]
            B = df[(df['timestamp']==tstp2)&(df['inst_token']=='vehicle_info')]['ego_pos'].iloc[0]
            AB =   [round(B[0] - A[0],3),round(B[1] - A[1],3)]
            ABn = round(AB[0]/np.sqrt((AB[0]*AB[0] + AB[1]*AB[1])),3),round(AB[1]/np.sqrt((AB[0]*AB[0] + AB[1]*AB[1])),3)
            if len(list_pos)!=0 :
                a = ABn[1]/ABn[0]
                c = - A[0] * a + A[1]
                dmin = 50
                posmin = []
                row = None
                for j in range(df_curr.shape[0]):
                    p = df_curr.iloc[j]['object_pos']
                    d = np.absolute(p[1] - a * p[0] - c)/ np.sqrt(a*a + 1)
                    distance_ego = round(np.linalg.norm(np.array(A) - np.array(p)),3)
                    if distance_ego < dmin and d < 5:
                        dmin = distance_ego
                        posmin = p
                        row = df_curr.iloc[j]
                if posmin != []:
                    return row
                
    for i in range(len(liste_temps)):
        row = compute_one_sample(i)
        new_df = new_df.append(row)
    #print(new_df.shape)
    #display(new_df)
    return new_df

def compute_near_vehicle_dataframe_all_scene():
    scenes = nusc.scene
    list_rows = []
    first = True
    i = 0
    for s in scenes:
        if s['name'] not in blacklist and s['name']  not in ["scene-0003","scene-0419"]:
            #scene_token = nusc.field2token('scene', 'name', s['name'])[0]
            #scene = nusc.get('scene',scene_token)
            #nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')
            df = build_dataframe_for_one_scene(s,False)
            if first:
                new_df = compute_near_vehicle_dataframe_one_scene(df)
                first = False
            else:
                return_df = compute_near_vehicle_dataframe_one_scene(df)
                new_df = new_df.append(return_df)
    return new_df
            
#init
scene_name = 'scene-0061'    
my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
scene = nusc.get('scene',my_scene_token)
#nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')

pos = df_ego[['timestamp','ego_pos','ego_speed']]    
#test = comput e_near_vehicle_dataframe_one_scene(df)
print(test.shape)
near_df = compute_near_vehicle_dataframe_all_scene()
print(near_df.shape)
near_df.to_csv("./data/near_dataframe_full.csv")

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  
%matplotlib qt


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
n = 100
ax.scatter(near_df['ego_speed'],near_df['brake'],near_df['distance'],marker='o')
ax.set_xlabel('V')
ax.set_ylabel('Brake')
ax.set_zlabel('Distance')
ax.set_ylim([0, 20])

plt.show()

In [ ]:
display(near_df)

Cette façon de faire avec distance et vitesse à l'instant t seulement n'est pas bonne. Je vais essayer mtn de rajouter ces informations du passé (t-1,-2...) pour avoir plus de paramètres, et j'espère avoir un résultat concret.

In [ ]:
near_df = pd.read_csv('./data/near_dataframe_full.csv')
near_df.shape

In [ ]:
# Pas opti (copy, à améliorer)
# Ajout de colonnes dans le dataframe pour les informations antérieurs (distance + vitesse)
def add_past_data_to_dataframe(dataframe,k_past_data):
    clmns = dataframe.columns.to_list()
    for j in range(1,k_past_data+1):
        clmns += ['distance_'+str(j)] + ['ego_speed_'+str(j)]
    new_df = pd.DataFrame(columns=clmns)
    set_scene = sorted(set(dataframe['scene'].to_list()))
    for s in set_scene:
        df_curr =  dataframe[dataframe['scene']==s]
        for i in range(k_past_data,df_curr.shape[0]):
            row = df_curr.iloc[i].copy()
            for j in range(1,k_past_data+1):
                row['distance_'+str(j)] = df_curr.iloc[i-j]['distance']
                row['ego_speed_'+str(j)] = df_curr.iloc[i-j]['ego_speed']
            new_df = new_df.append(row)
    #display(new_df)
    print(new_df.shape)
    return new_df
                       
    

train_data = add_past_data_to_dataframe(near_df,5)
features = [col for col in train_data.columns.to_list() if ('distance' in col or'ego_speed' in col)]
print(train_data.columns,features)

In [ ]:
[0] * 5